<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-27 11:01:11
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.33 C
CY Investment:  1.60 C
Reserve:  2.44 L
Current:  1.37 C
-------------------
Today PnL: -1.02 L (-0.73%)
Current PnL: -37.28 L (-23.31%)
CY Booked + Current PnL: -21.60 L (-13.5%)
-------------------
Total profit:  1.31 L
Total loss:  -38.60 L
-------------------
Total Booked + Current PnL: 5.38 L (4.04%)
Total Booked PnL: 42.66 L (32.07%)
Curr Year Booked PnL: 15.68 L (11.43%)
Prev Year Booked PnL: 26.98 L (20.29%)
Est FTT:  2.41 C
Est FTT PnL: 1.04 C (75.75%)
Deployed:  1.33 C
Current:  1.37 C
CAGR/XIRR %: 1.56%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COALINDIA,-1.35,5.16,11.89,17.67,18814.0,7770.0,158230.0,478.51,26.26,54.0,L-LC,11.00,185.0,0.41,1.17,27.02,XY25,ATH,MINING
50,MASFIN,-2.92,0.73,20.68,21.57,20411.0,720.0,98700.0,397.04,-13.28,50.0,H-SC,6.64,167.0,0.04,0.73,44.03,XR,ATH,FINANCE
77,TTKPRESTIG,-0.32,-31.69,46.53,0.09,32031.0,-31937.0,68840.0,770.00,59.87,31.0,M-SC,11.73,250.0,-1.00,0.51,0.00,OX40N,NTT,DURABLES
51,MEDANTA,-1.06,-4.03,29.28,24.08,35674.0,-5114.0,121836.0,1486.00,-4.72,52.0,X-SC,7.95,87.0,-0.14,0.90,11.59,XY24,NTT,HEALTHCARE
37,ICICIGI,-1.78,1.27,17.84,19.34,36153.0,2542.0,202651.0,2252.93,-14.93,50.0,X-MC,1.08,66.0,0.07,1.50,17.41,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
24,DIXON,2.07,-28.41,83.36,31.26,137711.0,-65568.0,165200.0,18931.72,-68.06,36.0,X-MC,24.18,54.0,-0.48,1.23,2.07,X40N,ATH,IT
26,EASEMYTRIP,2.00,-52.10,204.15,45.70,172913.0,-92116.0,84699.0,26.40,-11.99,62.0,M-SC,18.41,204.0,-0.53,0.63,39.55,XY24,NTT,TRAVEL
27,FINCABLES,1.91,12.72,84.44,107.91,140533.0,18786.0,166430.0,1641.55,1.47,72.0,M-SC,7.14,220.0,0.13,1.24,25.21,OX40N,ATH,CABLES
49,LAOPALA,1.62,-37.89,123.93,39.09,100921.0,-49671.0,81434.0,464.00,77.89,58.0,H-SC,13.40,154.0,-0.49,0.60,8.90,AR,NTT,CERAMICS
15,BSOFT,1.60,-29.31,113.31,50.79,112657.0,-41229.0,99424.0,831.70,-9.16,45.0,H-SC,5.25,172.0,-0.37,0.74,15.76,XR,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ANGELONE,-4.14,-90.32,1183.54,24.20,349594.0,-275712.0,29538.0,3033.00,-85.87,36.0,X-SC,2.46,97.0,-0.79,0.22,23.70,X40N,NTT,FINANCE
17,CAMS,-3.39,-10.30,38.77,24.48,87588.0,-25938.0,225918.0,950.00,-14.13,35.0,X-SC,5.65,89.0,-0.30,1.68,11.01,X40N,NTT,MISC
11,BANDHANBNK,-3.09,-13.25,121.80,92.42,294337.0,-36903.0,241656.0,400.00,107.61,68.0,H-SC,8.06,174.0,-0.13,1.79,33.12,XY24,NTT,BANKS
55,RAJESHEXPO,-3.04,-71.72,253.83,0.07,98476.0,-98381.0,38796.0,518.00,1321.72,31.0,L-SC,9.90,268.0,-1.00,0.29,0.00,OX40N,NTT,JEWELLERY
50,MASFIN,-2.92,0.73,20.68,21.57,20411.0,720.0,98700.0,397.04,-13.28,50.0,H-SC,6.64,167.0,0.04,0.73,44.03,XR,ATH,FINANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,-2.92,0.73,20.68,21.57,20411.0,720.0,98700.0,397.04,-13.28,50.0,H-SC,6.64,167.0,0.04,0.73,44.03,XR,ATH,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
27,FINCABLES,1.91,12.72,84.44,107.91,140533.0,18786.0,166430.0,1641.55,1.47,72.0,M-SC,7.14,220.0,0.13,1.24,25.21,OX40N,ATH,CABLES
66,SIS,-0.25,-30.84,75.33,21.26,57848.0,-34239.0,76793.0,528.00,1803.11,38.0,H-SC,5.59,168.0,-0.59,0.57,5.87,OX40N,NTT,MISC
77,TTKPRESTIG,-0.32,-31.69,46.53,0.09,32031.0,-31937.0,68840.0,770.00,59.87,31.0,M-SC,11.73,250.0,-1.00,0.51,0.00,OX40N,NTT,DURABLES
41,INDIGOPNTS,-0.74,-33.19,49.71,0.02,57973.0,-57937.0,116622.0,1408.00,78.01,27.0,M-SC,12.18,222.0,-1.00,0.87,1.44,OX40N,NTT,PAINTS
48,KANSAINER,-0.42,-33.40,70.37,13.47,126387.0,-90063.0,179604.0,340.00,-42.69,32.0,H-SC,13.08,162.0,-0.71,1.33,0.96,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VOLTAS,0.82,21.82,19.19,45.2,44824.0,41838.0,233580.0,1856.0,39.43,65.0,X-MC,9.07,79.0,0.93,1.73,30.1,XY25,NTT,AC


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COALINDIA,-1.35,5.16,11.89,17.67,18814.0,7770.0,158230.0,478.51,26.26,54.0,L-LC,11.00,185.0,0.41,1.17,27.02,XY25,ATH,MINING
27,FINCABLES,1.91,12.72,84.44,107.91,140533.0,18786.0,166430.0,1641.55,1.47,72.0,M-SC,7.14,220.0,0.13,1.24,25.21,OX40N,ATH,CABLES
50,MASFIN,-2.92,0.73,20.68,21.57,20411.0,720.0,98700.0,397.04,-13.28,50.0,H-SC,6.64,167.0,0.04,0.73,44.03,XR,ATH,FINANCE
1,ABB,-0.84,11.58,30.46,45.57,88916.0,30293.0,291912.0,7934.00,-28.26,66.0,H-MC,8.81,125.0,0.34,2.17,29.63,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,-0.84,11.58,30.46,45.57,88916.0,30293.0,291912.0,7934.00,-28.26,66.0,H-MC,8.81,125.0,0.34,2.17,29.63,AR,NTT,ELECTRICAL
27,FINCABLES,1.91,12.72,84.44,107.91,140533.0,18786.0,166430.0,1641.55,1.47,72.0,M-SC,7.14,220.0,0.13,1.24,25.21,OX40N,ATH,CABLES
50,MASFIN,-2.92,0.73,20.68,21.57,20411.0,720.0,98700.0,397.04,-13.28,50.0,H-SC,6.64,167.0,0.04,0.73,44.03,XR,ATH,FINANCE
85,ZYDUSLIFE,-1.41,-2.43,39.01,35.63,79768.0,-5094.0,204480.0,1286.17,-14.86,56.0,H-MC,4.88,120.0,-0.06,1.52,13.84,AR,ATH,PHARMA
40,INDIAMART,-1.18,-7.39,123.22,106.72,140742.0,-9116.0,114220.0,4810.62,-58.20,42.0,H-SC,8.08,140.0,-0.06,0.85,14.26,AR,ATH,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,-0.04,-38.63,156.15,57.19,312506.0,-125995.0,200132.0,485.00,-64.88,18.0,X-SC,18.82,81.0,-0.40,1.49,0.00,XY24,NTT,FMCG
43,INFY,1.10,-11.28,51.31,34.24,142438.0,-35285.0,277603.0,1972.00,-30.33,25.0,X-LC,8.69,3.0,-0.25,2.06,2.18,X40,NTT,IT
12,BATAINDIA,-0.64,-49.07,165.47,35.20,108437.0,-63137.0,65533.0,2096.00,-14.08,27.0,X-SC,22.33,91.0,-0.58,0.49,0.00,X40,NTT,FOOTWEAR
53,PGHH,-0.25,-14.39,54.21,32.01,99087.0,-30736.0,182784.0,17616.87,-42.16,30.0,X-MC,9.59,57.0,-0.31,1.36,0.00,X40,ATH,FMCG
34,HCLTECH,1.37,-9.64,37.05,23.83,80988.0,-23333.0,218591.0,1908.19,-4.88,31.0,X-LC,6.06,8.0,-0.29,1.62,4.88,X40,ATH,IT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,ICICIGI,-1.78,1.27,17.84,19.34,36153.0,2542.0,202651.0,2252.93,-14.93,50.0,X-MC,1.08,66.0,0.07,1.50,17.41,X40,ATH,INSURANCE
78,UNITDSPR,-1.16,0.84,19.76,20.77,46654.0,1960.0,236104.0,1644.00,-10.63,46.0,X-MC,1.26,64.0,0.04,1.75,8.30,X40N,NTT,BREWERIES
22,DABUR,-1.90,0.59,42.86,43.70,106949.0,1464.0,249532.0,735.00,-8.76,51.0,X-MC,1.43,70.0,0.01,1.85,15.73,XY24,BTT,FMCG
59,RELIANCE,-0.38,-10.04,39.28,25.30,71016.0,-20169.0,180794.0,1952.00,-12.34,42.0,X-LC,2.29,21.0,-0.28,1.34,21.10,XY25,BTT,REFINERIES
5,ANGELONE,-4.14,-90.32,1183.54,24.20,349594.0,-275712.0,29538.0,3033.00,-85.87,36.0,X-SC,2.46,97.0,-0.79,0.22,23.70,X40N,NTT,FINANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
2,ABBOTINDIA,-0.80,-12.29,34.61,18.07,54293.0,-21985.0,156870.0,35195.0,-25.27,35.0,X-MC,11.52,58.0,-0.40,1.16,0.0,X40,ATH,PHARMA
3,ACC,-1.17,-33.00,145.01,64.15,231175.0,-78531.0,159420.0,3906.0,-49.40,33.0,X-SC,9.50,85.0,-0.34,1.18,0.0,XY24,BTT,CEMENT
9,BAJAJHFL,-0.74,-25.54,108.81,55.47,197100.0,-62145.0,181141.0,181.5,-36.04,32.0,X-MC,17.21,60.0,-0.32,1.34,0.0,X40N,ATH,FINANCE
8,AWL,-0.04,-38.63,156.15,57.19,312506.0,-125995.0,200132.0,485.0,-64.88,18.0,X-SC,18.82,81.0,-0.40,1.49,0.0,XY24,NTT,FMCG
12,BATAINDIA,-0.64,-49.07,165.47,35.20,108437.0,-63137.0,65533.0,2096.0,-14.08,27.0,X-SC,22.33,91.0,-0.58,0.49,0.0,X40,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ANGELONE,-4.14,-90.32,1183.54,24.20,349594.0,-275712.0,29538.0,3033.0,-85.87,36.0,X-SC,2.46,97.0,-0.79,0.22,23.70,X40N,NTT,FINANCE
54,QUESS,-1.12,-35.30,120.78,42.84,50799.0,-22947.0,42059.0,424.0,-57.58,35.0,X-SC,17.13,80.0,-0.45,0.31,1.17,XY24,NTT,MISC
58,RELAXO,-0.85,-54.52,234.90,52.31,155072.0,-79144.0,66016.0,1176.0,-51.60,36.0,X-SC,14.79,92.0,-0.51,0.49,0.00,X40N,NTT,FOOTWEAR
12,BATAINDIA,-0.64,-49.07,165.47,35.20,108437.0,-63137.0,65533.0,2096.0,-14.08,27.0,X-SC,22.33,91.0,-0.58,0.49,0.00,X40,NTT,FOOTWEAR
51,MEDANTA,-1.06,-4.03,29.28,24.08,35674.0,-5114.0,121836.0,1486.0,-4.72,52.0,X-SC,7.95,87.0,-0.14,0.90,11.59,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
45,ITC,-0.85,-16.87,43.22,19.06,122217.0,-57388.0,282778.0,452.00,-54.31,40.0,X-LC,17.19,1.0,-0.47,2.10,2.39,X40,NTT,FMCG
73,TCS,-0.07,-24.16,62.95,23.58,206534.0,-104537.0,328092.0,4311.58,-34.12,31.0,X-LC,9.13,2.0,-0.51,2.44,2.81,X40,ATH,IT
43,INFY,1.10,-11.28,51.31,34.24,142438.0,-35285.0,277603.0,1972.00,-30.33,25.0,X-LC,8.69,3.0,-0.25,2.06,2.18,X40,NTT,IT
75,TMPV,-0.65,-13.56,54.24,33.33,152759.0,-44164.0,281636.0,600.00,-81.44,64.0,X-LC,3.96,4.0,-0.29,2.09,15.14,XY24,NTT,AUTO
34,HCLTECH,1.37,-9.64,37.05,23.83,80988.0,-23333.0,218591.0,1908.19,-4.88,31.0,X-LC,6.06,8.0,-0.29,1.62,4.88,X40,ATH,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,INDUSINDBK,-0.49,-29.13,87.52,32.89,47886.0,-22492.0,54714.0,1800.00,-357.70,65.0,L-MC,12.11,258.0,-0.47,0.41,50.68,XR,NTT,BANKS
67,SONACOMS,-1.71,-7.27,49.99,39.09,46895.0,-7350.0,93809.0,804.02,-26.55,60.0,M-MC,10.84,193.0,-0.16,0.70,32.71,AR,ATH,AUTO
50,MASFIN,-2.92,0.73,20.68,21.57,20411.0,720.0,98700.0,397.04,-13.28,50.0,H-SC,6.64,167.0,0.04,0.73,44.03,XR,ATH,FINANCE
19,COALINDIA,-1.35,5.16,11.89,17.67,18814.0,7770.0,158230.0,478.51,26.26,54.0,L-LC,11.00,185.0,0.41,1.17,27.02,XY25,ATH,MINING
27,FINCABLES,1.91,12.72,84.44,107.91,140533.0,18786.0,166430.0,1641.55,1.47,72.0,M-SC,7.14,220.0,0.13,1.24,25.21,OX40N,ATH,CABLES


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,EASEMYTRIP,2.00,-52.10,204.15,45.70,172913.0,-92116.0,84699.0,26.40,-11.99,62.0,M-SC,18.41,204.0,-0.53,0.63,39.55,XY24,NTT,TRAVEL
1,ABB,-0.84,11.58,30.46,45.57,88916.0,30293.0,291912.0,7934.00,-28.26,66.0,H-MC,8.81,125.0,0.34,2.17,29.63,AR,NTT,ELECTRICAL
7,ATULAUTO,-0.82,-13.99,69.24,45.57,117402.0,-27574.0,169558.0,844.00,3672.97,59.0,M-SC,6.69,248.0,-0.23,1.26,28.32,XY24,NTT,AUTO
27,FINCABLES,1.91,12.72,84.44,107.91,140533.0,18786.0,166430.0,1641.55,1.47,72.0,M-SC,7.14,220.0,0.13,1.24,25.21,OX40N,ATH,CABLES
11,BANDHANBNK,-3.09,-13.25,121.80,92.42,294337.0,-36903.0,241656.0,400.00,107.61,68.0,H-SC,8.06,174.0,-0.13,1.79,33.12,XY24,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.82
1,25,45.86
2,50,77.82


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.00
MC    30.77
LC    25.25
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.55
X40      24.24
X40N     12.92
AR        9.33
XY25      9.10
XR        8.68
OX40N     7.35
SR        0.85
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    23.12
H-SC    22.70
X-LC    21.81
M-SC    11.03
X-SC     9.55
H-MC     5.11
M-MC     2.13
H-LC     1.23
L-LC     1.17
M-LC     1.04
L-SC     0.72
L-MC     0.41
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.97,-14.73,53.53
IT,11.62,-40.21,110.48
FINANCE,11.22,-38.16,92.53
MISC,6.96,-38.75,95.50
ELECTRICAL,6.50,-7.32,46.47
PAINTS,5.11,-34.03,53.93
INSURANCE,5.00,-0.65,35.09
PHARMA,4.21,-3.48,36.06
AUTO,4.05,-14.51,58.18


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3418756.0,21
X40,1453327.0,15
X40N,1380433.0,10
XR,1321642.0,12
AR,1315778.0,10
OX40N,803931.0,10
XY25,391105.0,6
SR,304988.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3734642.0,24
X-SC,1517273.0,10
M-SC,1436312.0,14
X-MC,1395756.0,15
X-LC,1333890.0,12
H-MC,391682.0,3
L-SC,193782.0,2
M-LC,151758.0,1
M-MC,99636.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1263986.0      6
           AR         956087.0      5
           XR         846088.0      7
M-SC       XY24       838163.0      6
X-LC       X40        709797.0      6
X-SC       X40N       701859.0      4
           XY24       630154.0      4
X-MC       X40        558270.0      7
           X40N       448438.0      4
H-SC       OX40N      363493.0      4
M-SC       OX40N      341962.0      5
H-SC       SR         304988.0      2
X-LC       XY24       265854.0      2
           X40N       230136.0      2
H-MC       XY24       222998.0      1
X-MC       XY24       197601.0      2
           XY25       191447.0      2
X-SC       X40        185260.0      2
M-SC       XR         180604.0      2
H-MC       AR         168684.0      2
M-LC       XR         151758.0      1
X-LC       XY25       128103.0      2
L-SC       OX40N       98476.0      1
           XR          95306.0      1
M-SC       AR          75583.0      1
H-LC       AR          68529.0      1
M-MC       XY25        52741.0      1
L-MC       XR          47886.0      1
M-MC       AR          46895.0      1
L-LC       XY25        18814.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
